In [1]:
#df_arranjosra Bibliotecas
import pandas as pd
import math

In [2]:
#Leitura parametros Iniciais
df_arranjos = pd.read_csv('D38999_arranjos.csv', ';')
df_crimp = pd.read_csv('D38999_crimp.csv', ';')

In [73]:
# Funções
# trataDF - Isola da menor bitola para maior

print(math.ceil(4.2))
def trataDF(df):
    seg = 1.3
    trata_df = df
    trata_df['22D'] = (trata_df['#28'] + trata_df['#26'] + trata_df['#24'] + trata_df['#22'])*1.5
    trata_df['#28'] = 0
    trata_df['#26'] = 0
    trata_df['#24'] = 0
    trata_df['#22'] = 0
    trata_df['20'] = trata_df['#20']
    trata_df['#20'] = 0
    trata_df['16'] = trata_df['#18'] + trata_df['#16']
    trata_df['12'] = trata_df['#14'] + trata_df['#12']
    trata_df = trata_df.drop(columns=['#28', '#26', '#24', '#22', '#20', '#18', '#16', '#14', '#12'])
    return trata_df

#Isola emissividade em lista recebendo um dataframe da interface
def isolaEmissividade(df_interface):
    emi = []
    emi_t =[['Emissividade', '#28', '#26', '#24','#22','#20','#18','#16','#14','#12']]
    for i in (df_interface['Emissividade'].unique()):
        emi = []
        emi.append(i)
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#28'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#26'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#24'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#22'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#20'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#18'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#16'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#14'].sum())
        emi.append(df_interface.loc[df_interface['Emissividade'] == i, '#12'].sum())
        emi_t.append(emi)
    return emi_t

def deletaLinha1(df):
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header
    return df

5


In [83]:
#Leitura Interface
df_interface = pd.read_csv('D38999_interface.csv', ';')

# Cria dataframe com emissividades com bitolas por emissividades isoladas
df = pd.DataFrame(isolaEmissividade(df_interface))

#Deleta primeira linha do dataframe
df = deletaLinha1(df)

#Isola bitolas em grupos
trata_df = trataDF(df)


In [84]:
#Classes

class preparaProcura:
    def __init__(self, trata_df):
        #Encontra Dataframe
        df_encontrado = pd.DataFrame(columns = ['Código', '22D','20','16','12','12(Coax)','8(coax)','8(Twinax)'])
        self.df_encontrado = df_encontrado
        
        # Lista os arranjos encontrados
        lista_arranjo = trata_df.values.tolist()
        self.listaArranjo = lista_arranjo
        
        #Compila os valores
        
        lista_total =[]
        lista_total = [['Emissividade', 'Bitola', 'Arranjo']]
        self.listaTotal = lista_total
        
        #Inicia busca
        self.tes = 0
        
    def incrementa(self):
        self.tes = self.tes + 1
        
    def zeraTes():
        self.tes = 0

class trataBitola:
    def __init__(self, bitola):
        if bitola == 22:
            self.bit = '22D'
            self.emis = 1
        if bitola == 20:
            self.bit = '20'
            self.emis = 2
        if bitola == 16:
            self.bit = '16'
            self.emis = 3 
        if bitola == 12:
            self.bit = '12'
            self.emis = 4                
            
        
class encontraArranjo:
    def __init__(self,prepP, bit, df_encontrado, df_arranjos):
        
        if (prepP.listaArranjo[prepP.tes][bit.emis]) > df_arranjos[bit.bit].max():
            trataArranjo = math.ceil((prepP.listaArranjo[prepP.tes][1])/df_arranjos[bit.bit].max())
            for i in range(trataArranjo-1):
                df_encontrado = df_arranjos.loc[df_arranjos[bit.bit] == (df_arranjos[bit.bit].max())]
                merda = df_encontrado['Código'].values.tolist()
                prepP.listaTotal.append([prepP.listaArranjo[prepP.tes][0],bit.bit, merda[0]])
                prepP.listaArranjo[prepP.tes][bit.emis] = prepP.listaArranjo[prepP.tes][bit.emis] - df_arranjos[bit.bit].max()
    
    #Trata valores em caso ser menor que 1 conector 
        if (prepP.listaArranjo[prepP.tes][bit.emis]) !=0: # Acho que tinha essa bitola? Achou errado, caro otário
            df_encontrado = df_arranjos.loc[df_arranjos[bit.bit] == (prepP.listaArranjo[prepP.tes][bit.emis])]
            for i in range(df_arranjos[bit.bit].max()): #Procura até a ultima configuração possível para está bitola
                if (df_encontrado.empty): # Se não encontrou, itera para um valor acima
                    prepP.listaArranjo[prepP.tes][bit.emis] = prepP.listaArranjo[prepP.tes][bit.emis] + 1;
                    df_encontrado = df_arranjos.loc[df_arranjos[bit.bit] == (prepP.listaArranjo[prepP.tes][bit.emis])]
                
                #Separa se pode estar junto a outras bitolas ou não
                    trataArranjo = df_encontrado.values.tolist()
                    if not df_encontrado.empty:
                        if (trataArranjo[0][4] >0) or (trataArranjo[0][5] >0) or (trataArranjo[0][6] >0) or (trataArranjo[0][7] >0):
                            prepP.listaArranjo[prepP.tes][bit.emis] = prepP.listaArranjo[prepP.tes][bit.emis] + 1;
                            df_encontrado = df_encontrado.iloc[0:0] #Esvazia o iloc
        
        # Adiciona a lista a bitola selecionada
            if not df_encontrado.empty:
                merda = df_encontrado['Código'].values.tolist()
                prepP.listaArranjo[prepP.tes].append(merda[0])
                prepP.listaTotal.append([prepP.listaArranjo[prepP.tes][0],bit.bit, merda[0]]) 



In [87]:
prepP = preparaProcura(trata_df)
prepP.tes = 0
df_encontrado = pd.DataFrame(columns = ['Código', '22D','20','16','12','12(Coax)','8(coax)','8(Twinax)'])

for i in prepP.listaArranjo:
    
    bit = trataBitola(22)
    precA = encontraArranjo(prepP, bit, df_encontrado, df_arranjos)
    bit = trataBitola(20)
    precA = encontraArranjo(prepP, bit, df_encontrado, df_arranjos)
    bit = trataBitola(16)
    precA = encontraArranjo(prepP, bit, df_encontrado, df_arranjos)
    bit = trataBitola(12)
    precA = encontraArranjo(prepP, bit, df_encontrado, df_arranjos)
    
        
    prepP.tes = prepP.tes + 1

df_list = pd.DataFrame(prepP.listaTotal)
df_list

,0,1,2
0,Emissividade,Bitola,Arranjo
1,A,22D,J35
2,A,22D,J35
3,A,22D,J35
4,A,20,B5
5,A,16,E8
6,A,12,C13
7,S,22D,A35
8,S,20,B5


## 

In [12]:
# Consulta rápida para todos arranjos de uma bitola
df_encontrado = df_arranjos.loc[df_arranjos['20'] >= 5]
df_encontrado

,Código,22D,20,16,12,12(Coax),8(coax),8(Twinax)
4,B5,0,5,0,0,0,0,0
7,B98,0,6,0,0,0,0,0
8,B99,0,7,0,0,0,0,0
10,C8,0,8,0,0,0,0,0
13,C98,0,10,0,0,0,0,0
16,D15,0,14,1,0,0,0,0
17,D18,0,18,0,0,0,0,0
18,D19,0,19,0,0,0,0,0
20,D97,0,8,4,0,0,0,0
25,E26,0,26,0,0,0,0,0


[]

## math.ceil(200/df_arranjos['22D'].max())